# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [39]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

from sklearn import grid_search
from sklearn import cross_validation
from sklearn import metrics

In [40]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [41]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]-1
n_passed = (student_data.passed == 'yes').sum()
n_failed = (student_data.passed == 'no').sum()
grad_rate = 100*n_passed/float(n_students)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [42]:
# Shuffling to avoid bias selecting when split data into training and test sets
student_data = student_data.iloc[np.random.permutation(len(student_data))]

# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows
print y_all.head()

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
    school sex  age address famsize Pstatus  Medu  Fedu      Mjob     Fjob  \
170     GP   M   16       U     GT3       T     3     4     other    other   
60      GP   F   16       R     GT3       T     4     4    health  teacher   
88      GP   M   16       U     GT3       T     2     2  services    other   
81      GP   M   15       U     GT3       T     2     3     other    other   
300     GP   F   18       U     LE3       A     4     4    health    other   

      ...    higher internet  romantic  famrel  freetime goout Dalc Walc  \
170   ...       yes      yes        no       3         4     5    2    4   
60    ...

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [43]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [44]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
def split_data(X_all, y_all, num_all, num_train) :
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, test_size=95, random_state=42)
    X_train = X_train[0:num_train]
    y_train = y_train[0:num_train]
    return [X_train, y_train, X_test, y_test]

[X_train, y_train, X_test, y_test] = split_data(X_all, y_all, num_all, num_train)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [45]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training KNeighborsClassifier...
Done!
Training time (secs): 0.001


In [46]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.862385321101


In [47]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.8


### Finding Optimal Complexity (elbow method & GridSearchCV)

We will plot the error with varying complexity (e.g. k value of KNN, or max_depth of decision tree)
For visualizing

In [48]:
def performance_metric(y_train, y_pred) :
    from sklearn import metrics
    
    return metrics.f1_score(y_train, y_pred, pos_label='yes')
    

In [49]:
def model_complexity(clf, X_train, y_train, X_test, y_test, complexity):
    train_score = np.zeros(len(complexity))
    test_score = np.zeros(len(complexity))
    
    for i, d in enumerate(complexity):
        if clf.__class__.__name__ == 'KNeighborsClassifier':
            clf.n_neighbors = d        
        if clf.__class__.__name__ == 'DecisionTreeClassifier':
            clf.max_depth = d
        if clf.__class__.__name__ == 'SVC':
            clf.C = d
        clf.fit(X_train, y_train)
        
        train_score[i] = performance_metric(y_train, clf.predict(X_train))
        test_score[i] = performance_metric(y_test, clf.predict(X_test))
        
    print "------------------------------------------"
    print train_score
    print test_score
        
    # Plot the model complexity graph
    #pl.figure(figsize=(7, 5))
    #pl.title('Classifier Complexity Performance : comparing result with varying complexity')
    #pl.plot(max_depth, test_err, lw=2, label = 'Testing Error')
    #pl.plot(max_depth, train_err, lw=2, label = 'Training Error')
    #pl.legend()
    #pl.xlabel('Maximum Depth')
    #pl.ylabel('Total Error')
    
    #pl.show()

In [50]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# Train using different training size
def train_for_different_size(clf, X_all, y_all, num_all, num_train):
    for num in num_train:
        [X_train, y_train, X_test, y_test] = split_data(X_all, y_all, num_all, num)
        train_predict(clf, X_train, y_train, X_test, y_test)
    
# (plotting)Visualize the score with varying complexity
clf = KNeighborsClassifier()
k_value = np.arange(1,10)
[X_train, y_train, X_test, y_test] = split_data(X_all, y_all, num_all, 300)
model_complexity(clf, X_train, y_train, X_test, y_test, complexity=k_value)

# Find the best complexity (using grid_search CV)
############ skip grid_search this time #############
#scoring_function = metrics.make_scorer(f1_score, pos_label='yes', greater_is_better = True)
#parameters = {'n_neighbors':np.arange(1,10)}
#clf = grid_search.GridSearchCV(clf, parameters, scoring_function)
#clf.fit(X_train, y_train)
#best_n_neighbors = clf.best_params_
#print clf.best_params_
#####################################################

# KNNClassifier
KNN_clf = KNeighborsClassifier()
num_train=[100,200,300]
train_for_different_size(KNN_clf, X_all, y_all, num_all, num_train)

# Train for different set_size = 200
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

------------------------------------------
[ 1.          0.84117647  0.88888889  0.84155844  0.86238532  0.8345679
  0.84454756  0.83173077  0.83218391]
[ 0.72727273  0.63157895  0.80555556  0.73282443  0.8         0.75
  0.7972973   0.79432624  0.7755102 ]
------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.000
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.866666666667
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.808219178082
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.825622775801
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test 

### Decision Tree Classifier

From now on we will use Decision Tree Classifier as a learning model

In [51]:
from sklearn.tree import DecisionTreeClassifier

# (plotting)Visualize the score with varying complexity
clf = DecisionTreeClassifier(random_state=0, criterion="entropy")
max_depth = np.arange(1,10)
[X_train, y_train, X_test, y_test] = split_data(X_all, y_all, num_all, 300)
model_complexity(clf, X_train, y_train, X_test, y_test, complexity=max_depth)

# Find the best complexity (using grid_search CV)
############ skip grid_search this time #############
#scoring_function = metrics.make_scorer(f1_score, pos_label='yes', greater_is_better = True)
#parameters = {'max_depth':np.arange(1,10)}
#clf = grid_search.GridSearchCV(clf, parameters, scoring_function)
#clf.fit(X_train, y_train)
#best_max_depth = clf.best_params_
#print clf.best_params_
#####################################################

# DecisionTree Classifier
DT_clf = DecisionTreeClassifier(max_depth=5)
num_train=[100,200,300]
train_for_different_size(DT_clf, X_all, y_all, num_all, num_train)


------------------------------------------
[ 0.80555556  0.80555556  0.82598608  0.82973621  0.85929648  0.89876543
  0.9151671   0.9346247   0.945     ]
[ 0.82758621  0.82758621  0.8137931   0.8028169   0.72463768  0.75714286
  0.71212121  0.75714286  0.76470588]
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.916666666667
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.797297297297
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.914285714286
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000

### Support Vector Machine - Support Vector Classifier

This time we will apply SVM.SVC model for learning.

In [52]:
from sklearn.svm import SVC

# (plotting)Visualize the score with varying complexity
clf = SVC()
C = np.arange(1,20)
[X_train, y_train, X_test, y_test] = split_data(X_all, y_all, num_all, 300)
model_complexity(clf, X_train, y_train, X_test, y_test, complexity=C)

# Find the best complexity (using grid_search CV)
############ skip grid_search this time #############
#scoring_function = metrics.make_scorer(f1_score, pos_label='yes', greater_is_better = True)
#param_grid = [{'C': list(np.arange(1,20)), 'kernel': ['linear']},
#              {'C': list(np.arange(1,20)), 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
#             ]
#clf = grid_search.GridSearchCV(clf, param_grid, scoring_function)
#clf.fit(X_train, y_train)
#best_param_grid = clf.best_params_
#print clf.best_params_
#####################################################

# SupportVector Classifier
SV_clf = SVC()
num_train=[100,200,300]
train_for_different_size(SV_clf, X_all, y_all, num_all, num_train)

------------------------------------------
[ 0.87305122  0.91841492  0.93809524  0.94915254  0.96568627  0.96805897
  0.97524752  0.9800995   0.98746867  0.98994975  0.99244332  0.99494949
  0.99494949  0.99746835  0.99746835  1.          1.          1.          1.        ]
[ 0.82894737  0.82432432  0.81944444  0.83333333  0.82517483  0.8
  0.79136691  0.8         0.79136691  0.79136691  0.79136691  0.79136691
  0.77697842  0.76811594  0.76811594  0.76811594  0.76811594  0.76811594
  0.76811594]
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.887417218543
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.832214765101
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time 

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

I used 3 train models which are KNN, DecisionTree, SVM-SVC.
These 3 models' performances are very similar(F1 score are in 7.8 to 8.1)

Training time and Prediction time cost also very low (all below 0.007).
Finding best_params on SVC model takes long time. However once it find best parameter, then training and predicting time of model is very fast.

So I choose a SVM-SVC model (which has the best final F1 score). Here are some pros and cons on SVC model
- It is very effective when the features' dimension is very high.
- Normally it works well in small data sets. (but our training set is small and high dimension(35), so it's useful)

I explained this algorithm in detail at my pdf file. (please refer it)


In [53]:
# TODO: Fine-tune your model and report the best F1 score
# Our Final model is SVC classifier!
best_clf = SVC()

# Find the best complexity (using grid_search CV)
scoring_function = metrics.make_scorer(f1_score, pos_label='yes', greater_is_better = True)
param_grid = [{'C': list(np.arange(1,20)), 'kernel': ['linear', 'sigmoid']},
              {'C': list(np.arange(1,20)), 'gamma': [0.5, 0.3, 0.1, 0.005, 0.001, 0.0001], 'kernel': ['rbf']}
             ]
best_clf = grid_search.GridSearchCV(clf, param_grid, scoring_function)
best_clf.fit(X_train, y_train)
best_param_grid = best_clf.best_params_
print best_clf.best_params_

best_clf = SVC(kernel=best_param_grid['kernel'], C=best_param_grid['C'], gamma=best_param_grid['gamma'])
train_predict(best_clf, X_train, y_train, X_test, y_test)

{'kernel': 'rbf', 'C': 11, 'gamma': 0.001}
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.812095032397
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.860927152318


#### As you can see above, the best_clf (which complexity is tuned) shows higher performance!